# Code-to-Code Comparison: Dinwoodie

### National Renewable Energy Laboratory

#### Rob Hammond

##### 5 December 2022

In [1]:
from copy import deepcopy
from time import perf_counter
from pprint import pprint

import yaml
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from wombat.core import Simulation
from wombat.core.library import DINWOODIE, load_yaml

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

In [2]:
# Converting Labor values to fixed cost input for the base case
tech_salary_annual = 80000
techs = 20
capacity = 240 * 1000  # 240 -> kW
f"{tech_salary_annual * techs / capacity:.4f}"

'6.6667'

In [3]:
configs = [
    "base",
    "more_ctvs",
    "fewer_ctvs",
    "more_techs",
    "fewer_techs",
    "failure_50",
    "failure_200",
    "no_hlvs",
    "no_weather",
    "historic_weather",
    "manual_resets_only",
    "minor_repairs_only",
    "medium_repairs_only",
    "major_repairs_only",
    "major_replacements_only",
    "annual_service_only",
]
columns = deepcopy(configs)
results = {
    "availability - time based": [],
    "availability - production based": [],
    "capacity factor - net": [],
    "capacity factor - gross": [],
    "power production": [],
    "task completion rate": [],
    "annual direct O&M cost": [],
    "annual vessel cost": [],
    "ctv cost": [],
    "fsv cost": [],
    "hlv cost": [],
    "annual repair cost": [],
    "annual technician cost": [],
    "ctv utilization": [],
    "fsv utilization": [],
    "hlv utilization": [],
    
}

In [4]:
for config in configs:
    # Run the simulation
    start = perf_counter()
    sim = Simulation(DINWOODIE, f"{config}.yaml")
    sim.run()
    end = perf_counter()
    print(f"{sim.config.name.rjust(34)} | {(end - start) / 60:.2f} m")
    
    # Gather the results of interest
    years = sim.metrics.events.year.unique().shape[0]
    mil = 1000000
    
    availability_time = sim.metrics.time_based_availability(frequency="project", by="windfarm").values[0][0]
    availability_production = sim.metrics.production_based_availability(frequency="project", by="windfarm").values[0][0]
    cf_net = sim.metrics.capacity_factor(which="net", frequency="project", by="windfarm").values[0][0]
    cf_gross = sim.metrics.capacity_factor(which="gross", frequency="project", by="windfarm").values[0][0]
    power_production = sim.metrics.power_production(frequency="project", by_turbine=False).values[0][0]
    completion_rate = sim.metrics.task_completion_rate(which="both", frequency="project").values[0][0]
    parts = sim.metrics.events[["materials_cost"]].sum().sum()
    techs = sim.metrics.project_fixed_costs(frequency="project", resolution="low").operations[0]
    total = sim.metrics.events[["total_cost"]].sum().sum()
    
    equipment = sim.metrics.equipment_costs(frequency="project", by_equipment=True)
    equipment_sum = equipment.sum().sum()
    hlv = equipment[[el for el in equipment.columns if "Heavy Lift Vessel" in el]].sum().sum()
    fsv = equipment[[el for el in equipment.columns if "Field Support Vessel" in el]].sum().sum()
    ctv = equipment[[el for el in equipment.columns if "Crew Transfer Vessel" in el]].sum().sum()
    
    utilization = sim.metrics.service_equipment_utilization(frequency="project")
    hlv_ur = utilization[[el for el in utilization.columns if "Heavy Lift Vessel" in el]].mean().mean()
    fsv_ur = utilization[[el for el in utilization.columns if "Field Support Vessel" in el]].mean().mean()
    ctv_ur = utilization[[el for el in utilization.columns if "Crew Transfer Vessel" in el]].mean().mean()
    
    # Log the results of interest
    results["availability - time based"].append(availability_time)
    results["availability - production based"].append(availability_production)
    results["capacity factor - net"].append(cf_net)
    results["capacity factor - gross"].append(cf_gross)
    results["power production"].append(power_production)
    results["task completion rate"].append(completion_rate)
    results["annual direct O&M cost"].append((total + techs) / mil / years)
    results["annual vessel cost"].append(equipment_sum / mil / years)
    results["ctv cost"].append(ctv / mil / years)
    results["fsv cost"].append(fsv / mil / years)
    results["hlv cost"].append(hlv / mil / years)
    results["annual repair cost"].append(parts / mil / years)
    results["annual technician cost"].append(techs / mil / years)
    results["ctv utilization"].append(ctv_ur)
    results["fsv utilization"].append(fsv_ur)
    results["hlv utilization"].append(hlv_ur)
    sim.env.cleanup_log_files(log_only=True)

                    dinwoodie_base | 1.22 m
               dinwoodie_more_ctvs | 1.07 m
              dinwoodie_fewer_ctvs | 1.14 m
              dinwoodie_more_techs | 1.31 m
             dinwoodie_fewer_techs | 1.27 m
              dinwoodie_failure_50 | 1.03 m
             dinwoodie_failure_200 | 1.57 m
                 dinwoodie_no_hlvs | 1.22 m
              dinwoodie_no_weather | 1.24 m
        dinwoodie_historic_weather | 1.06 m
      dinwoodie_manual_resets_only | 0.88 m
      dinwoodie_minor_repairs_only | 0.80 m
     dinwoodie_medium_repairs_only | 0.65 m
      dinwoodie_major_repairs_only | 0.65 m
 dinwoodie_major_replacements_only | 0.63 m
     dinwoodie_annual_service_only | 0.78 m


In [5]:
# Save the results
with open(DINWOODIE / "outputs" / "results_dict_v0.5.2.yaml", "w") as f:
    yaml.dump(results, f, default_flow_style=False, sort_keys=False)

# dataframe/csv format
results_df = pd.DataFrame(results.values(), columns=columns, index=results.keys()).fillna(0)
results_df.to_csv(DINWOODIE / "outputs" / "results_data_v0.5.2.csv", index_label="result")

In [6]:
results_df

,base,more_ctvs,fewer_ctvs,more_techs,fewer_techs,failure_50,failure_200,no_hlvs,no_weather,historic_weather,manual_resets_only,minor_repairs_only,medium_repairs_only,major_repairs_only,major_replacements_only,annual_service_only
availability - time based,0.95,0.94,0.94,0.93,0.94,0.95,0.78,0.97,0.96,0.94,0.99,0.99,1.00,1.00,0.97,0.98
availability - production based,0.95,0.94,0.94,0.93,0.94,0.95,0.78,0.97,0.00,0.94,0.99,0.99,1.00,1.00,0.97,0.98
capacity factor - net,0.45,0.44,0.45,0.44,0.45,0.45,0.37,0.46,0.00,0.45,0.47,0.47,0.47,0.47,0.46,0.47
capacity factor - gross,0.48,0.47,0.47,0.47,0.47,0.47,0.47,0.47,0.00,0.48,0.47,0.47,0.47,0.47,0.47,0.47
power production,"9,489,372,788.00","8,954,108,979.50","9,034,137,776.00","8,917,795,016.50","8,986,550,083.50","9,066,280,524.50","7,457,227,765.50","9,318,042,176.50",0.00,"7,623,393,085.00","9,520,006,080.00","9,523,465,753.50","9,558,341,119.00","9,569,349,929.00","9,314,968,292.50","9,429,487,273.50"
task completion rate,1.00,1.00,1.00,1.00,1.00,1.00,0.99,1.00,1.00,1.00,1.00,1.00,1.00,0.95,0.95,0.92
annual direct O&M cost,13.80,19.92,14.12,15.91,15.47,9.82,18.92,5.17,13.90,15.90,3.26,3.74,3.96,4.80,12.45,4.50
annual vessel cost,8.80,12.11,8.78,9.82,10.63,5.78,12.52,1.75,8.71,10.37,1.73,1.81,1.84,1.99,7.94,1.74
ctv cost,1.83,3.07,1.75,1.79,1.77,1.75,1.76,1.75,1.91,1.79,1.73,1.81,1.84,1.74,1.61,1.74
fsv cost,0.22,0.16,0.24,0.31,0.24,0.09,0.22,0.00,0.19,0.26,0.00,0.00,0.00,0.25,0.00,0.00
